In [1]:
import os
import cv2
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [3]:
import tensorflow as tf

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import save_model, load_model

from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Dense, Flatten, Dropout
from keras.layers import LSTM, GRU, Bidirectional

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam



In [4]:
def create_dataset():
    X=[]
    y=[]

    folders = ['backhand', 'forehand', 'service', 'smash']
    for folder in folders:
        base_path = f'../keypoints/{folder}/'
        for csv_path in os.listdir(base_path):
            data = pd.read_csv(base_path + csv_path)
            
            features = data.loc[:, data.columns != 'class']

            X.append(features.to_numpy())
            y.append(data["class"].iloc[0])

    X = np.stack(X, axis=0)
    X = np.array(X)
    y = np.array(y)

    return X, y

In [5]:
# 데이터 좌우 대칭 -> x 값만 변화, y값 변화 없음
def revert_datasets(df):
    revert_data = df.copy()
    
    for feature in df.columns:
        if feature[-2:] == '_x':
            revert_data[feature] = 1 - df[feature]
    return revert_data

In [7]:
sequence_length = 40

def create_revert_dataset():
    X=[]
    y=[]

    folders = ['backhand', 'forehand', 'service', 'smash']
    for folder in folders:
        base_path = f'../keypoints/{folder}/'
        for csv_path in os.listdir(base_path):
            df = pd.read_csv(base_path + csv_path)
            
            # 60개씩 끊어서 저장
            num_rows = df.shape[0]
            num_batches = num_rows // sequence_length

            for i in range(num_batches):
                start_idx = i * sequence_length
                end_idx = (i + 1) * sequence_length

                data = df.iloc[start_idx : end_idx, :].reset_index(drop=True)

                # 데이터 변환
                # revert_data = revert_datasets(df)

                # X.append(revert_data.values[1:])
                # y.append(revert_data['class'].values[0])

                features = data.loc[:, data.columns != 'class']
                X.append(features.to_numpy())
                y.append(data["class"].iloc[0])
                
    X = np.stack(X, axis=0)

    X = np.array(X)
    y = np.array(y)

    return X, y

In [8]:
X, y = create_revert_dataset()

In [9]:
print(X.shape)
print(y.shape)

(904, 40, 34)
(904,)


In [10]:
# Split dataset
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, shuffle=True)

In [11]:
# One-hot Encoding
le = preprocessing.LabelEncoder()

y_train = le.fit_transform(y_train)
y_valid = le.fit_transform(y_valid)

num_classes = 4
y_train = to_categorical(y_train, num_classes=num_classes)
y_valid = to_categorical(y_valid, num_classes=num_classes)

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(632, 40, 34)
(632, 4)
(272, 40, 34)
(272, 4)


In [15]:
# Model 정의
model = Sequential([
    GRU(24, dropout=0.1, input_shape=(sequence_length, 34)),
    Dropout(0.2),
    Dense(8, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 24)                4320      
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 200       
                                                                 
 dense_3 (Dense)             (None, 4)                 36        
                                                                 
Total params: 4,556
Trainable params: 4,556
Non-trainable params: 0
_________________________________________________________________


In [17]:
# checkpoint = ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=0, save_best_only=True, mode='auto')
checkpoint = ModelCheckpoint('../models/model.h5', verbose=0, save_best_only=True)
scheduler = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=5, verbose=1, mode='auto')

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_valid, y_valid),
    epochs=300,
    callbacks=[checkpoint, scheduler]
)

# 모델 저장
path = '../models/model1_seq' + str(sequence_length) + '.h5'
save_model(model, path)

Epoch 1/300
20/20 [==============================] - 3s 36ms/step - loss: 1.3576 - acc: 0.3244 - val_loss: 1.2803 - val_acc: 0.3640 - lr: 0.0010
Epoch 2/300
20/20 [==============================] - 0s 11ms/step - loss: 1.3187 - acc: 0.3117 - val_loss: 1.2616 - val_acc: 0.4007 - lr: 0.0010
Epoch 3/300
20/20 [==============================] - 0s 9ms/step - loss: 1.2978 - acc: 0.3180 - val_loss: 1.2559 - val_acc: 0.3640 - lr: 0.0010
Epoch 4/300
20/20 [==============================] - 0s 7ms/step - loss: 1.2872 - acc: 0.3259 - val_loss: 1.2593 - val_acc: 0.4301 - lr: 0.0010
Epoch 5/300
20/20 [==============================] - 0s 9ms/step - loss: 1.2733 - acc: 0.3623 - val_loss: 1.2523 - val_acc: 0.4118 - lr: 0.0010
Epoch 6/300
20/20 [==============================] - 0s 8ms/step - loss: 1.2777 - acc: 0.3481 - val_loss: 1.2510 - val_acc: 0.4449 - lr: 0.0010
Epoch 7/300
20/20 [==============================] - 0s 8ms/step - loss: 1.2672 - acc: 0.3734 - val_loss: 1.2484 - val_acc: 0.4559 - l

In [ ]:
# 모델 불러오기
loaded_model = load_model('../models/model1.h5')

In [18]:
model = Sequential([
    LSTM(90, input_shape=(sequence_length, 34), return_sequences=True),
    LSTM(90, return_sequences=True),
    LSTM(90, return_sequences=False),
    Dense(8, activation='tanh', kernel_regularizer=l2(0.001)),
    Dense(4, activation='softmax')
])

In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 40, 90)            45000     
                                                                 
 lstm_1 (LSTM)               (None, 40, 90)            65160     
                                                                 
 lstm_2 (LSTM)               (None, 90)                65160     
                                                                 
 dense_4 (Dense)             (None, 8)                 728       
                                                                 
 dense_5 (Dense)             (None, 4)                 36        
                                                                 
Total params: 176,084
Trainable params: 176,084
Non-trainable params: 0
_________________________________________________________________


In [20]:
checkpoint = ModelCheckpoint('../models/model.h5', verbose=0, save_best_only=True)
scheduler = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=5, verbose=1, mode='auto')

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_valid, y_valid),
    epochs=300,
    callbacks=[checkpoint, scheduler]
)

# 모델 저장
path = '../models/model2_seq' + str(sequence_length) + '.h5'
save_model(model, path)

Epoch 1/300
20/20 [==============================] - 5s 70ms/step - loss: 1.3166 - acc: 0.3623 - val_loss: 1.2671 - val_acc: 0.3640 - lr: 0.0010
Epoch 2/300
20/20 [==============================] - 0s 18ms/step - loss: 1.2906 - acc: 0.3434 - val_loss: 1.2551 - val_acc: 0.4596 - lr: 0.0010
Epoch 3/300
20/20 [==============================] - 0s 18ms/step - loss: 1.2674 - acc: 0.3940 - val_loss: 1.2326 - val_acc: 0.3713 - lr: 0.0010
Epoch 4/300
20/20 [==============================] - 0s 18ms/step - loss: 1.2064 - acc: 0.4557 - val_loss: 1.1373 - val_acc: 0.5000 - lr: 0.0010
Epoch 5/300
20/20 [==============================] - 0s 17ms/step - loss: 1.2783 - acc: 0.4241 - val_loss: 1.2009 - val_acc: 0.4154 - lr: 0.0010
Epoch 6/300
20/20 [==============================] - 0s 17ms/step - loss: 1.2035 - acc: 0.5174 - val_loss: 1.1487 - val_acc: 0.5257 - lr: 0.0010
Epoch 7/300
20/20 [==============================] - 0s 16ms/step - loss: 1.1473 - acc: 0.5222 - val_loss: 1.1376 - val_acc: 0.536